# Introduction

**Kor** is a thin wrapper on top of LLMs that helps to extract structured data using LLMs. 

To use Kor, specify the schema of what should be extracted and provide some extraction examples.

As you're looking through this tutorial, examine 👀 the outputs carefully to understand what errors are being made.
Extraction isn't perfect, so understand the limitations of this approach before adopting it for your use case.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pprint

In [3]:
sys.path.insert(0, "../../")

In [4]:
from kor.extraction import Extractor
from kor.nodes import Object, Text, Number
from kor.llms import OpenAIChatCompletion, OpenAICompletion

## LLM

For the initial release, only OpenAI wrappers have been included with the package.

In [5]:
llm = OpenAIChatCompletion(model="gpt-3.5-turbo")

You can alternatively use a something like `text-davinci-003`, using `llm = OpenAICompletion(model='text-davinci-003')`.

----------
Create an extractor instance. The extactor is responsible for generating a prompt, feeding it into the LLM and parsing out the output.

In [6]:
model = Extractor(llm)

## Schema

Kor requires that you specify the `schema` of what you want parsed with some optional examples.

We'll start off by specifying a **very simple** schema:

In [7]:
schema = Text(
    id="first_name",
    description="The first name of a person",
    examples=[("I am billy.", "billy"), ("John Smith is 33 years old", "John")],
)

The schema above consists of a single text node (i.e., a single text input). 

The node will capture mentiosn of **first name** from a segment of text.

As part of the schema, we specified a `description` of what we're extracting, as well as 2 examples.

Including both a `description` and `examples` will likely improve performance.

## Extract

With a `model` and a `schema` defined, we're ready to extract data.

In [8]:
model("My name is Tom. I am a cat. My best friend is Bobby. He is not a cat.", schema)

{'first_name': ['Tom']}

In [9]:
model("My name is My name is My name is WOW.", schema)

{'first_name': ['WOW']}

In [10]:
# Note the extraction here. It's unlikely to be reasonable.
model("My name is My name is My name is MOO MOO.", schema)

{'first_name': ['MOO', 'MOO', 'MOO']}

In [11]:
model(
    (
        "My name is Bobby. My brother's name is the same as mine except that it starts"
        " with a `C`."
    ),
    schema,
)

{'first_name': ['Bobby', 'Cobby']}

In [12]:
model("My name is Bobby. My brother's name rhymes with mine.", schema)

{'first_name': ['Bobby']}

In [13]:
model("My name is Bobby. My brother's name is like mine but different.", schema)

{'first_name': ['Bobby']}

In [14]:
model(
    (
        "My name is Bobby. My brother's name is the same as mine but it does not have"
        " vowels."
    ),
    schema,
)

{'first_name': ['Bobby']}

## The Prompt

And here's the actual prompt that was sent to the LLM.

In [15]:
print(model.prompt_generator.format_as_string("user input goes here", schema))

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

{
 first_name: string[] // The first name of a person
}
```


For Union types the output must EXACTLY match one of the members of the Union type.

Please enclose the extracted information in HTML style tags with the tag name corresponding to the corresponding component ID. Use angle style brackets for the tags ('>' and '<'). Only output tags when you're confident about the information that was extracted from the user's query. If you can extract several pieces of relevant information from the query, then include all of them. If the type is an array, please repeat the corresponding tag name multiple times once for each relevant extraction. 

Input: I am billy.
Output: <first_name>billy</first_name>
Input: John 